In [28]:
#instaloader : instagram scraper
from instaloader import Instaloader, Profile
import pandas as pd
import random
from re import sub, match
from decimal import Decimal

#selenium, web scraper
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
from time import sleep

In [29]:
#instaloader (scrape data from instagram)

#login into instagram account
def login_instaloader(username, password):
    L = Instaloader()
    L.login(username, password)
    return L

#get follower
def get_followers_following(L,user):
    """
    parameter
    L    : instaloader object
    user : an account for which data will be taken (u can use your own account / other public account)
    
    return
    list of our followers, list of account we followed, account that doesn't follback and account we don't follback
    """
    profile = Profile.from_username(L.context, user)
    follower = [user.username for user in profile.get_followers()]
    follow = [user.username for user in profile.get_followees()]
    
    #save in dataframe
    followers = pd.DataFrame()
    followers['account'] = follower
    follows = pd.DataFrame()
    follows['account'] = follow
    
    #not follback
    not_follback = follows[~follows['account'].isin(follower)]
    #fans
    fans = followers[~followers['account'].isin(follow)]
    
    print("Account: "+user+" with "+str(len(follower))+" followers, "+str(len(follow))+" follow, "+str(not_follback.shape[0])+" doesn't follback and "+str(fans.shape[0])+" fans")
    return followers, follows, not_follback, fans

In [30]:
#selenium + webdriver (automatic operation)
class InstagramBot:
    
    #InstagramBot constructor (initiation for ig_account, ig_pass and webdriver)
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.driver = webdriver.Chrome()
    
    def close_browser(self):
        self.driver.close()
        
    def login(self):
        driver = self.driver
        driver.get('https://instagram.com')
        sleep(2)
        
        #fill the username field
        username_elm = driver.find_element_by_xpath("//input[@name='username']")
        username_elm.clear()
        username_elm.send_keys(self.username)
        
        #fill the password field
        password_elm = driver.find_element_by_xpath("//input[@name='password']")
        password_elm.clear()
        password_elm.send_keys(self.password)
        #press enter
        password_elm.send_keys(Keys.RETURN)
        sleep(4)
        
        #click not now
        driver.find_element_by_xpath("//button[@class='sqdOP yWX7d    y3zKF     ']").click()
        sleep(4)
        
        #click not now
        driver.find_element_by_xpath("//button[@class='aOOlW   HoLwm ']").click()
        sleep(2)
    
    def like_comment(self, account, comments, method=1):
        """
        account : ig_account that we will like+comment its post (str)
        comment : what comment we want to send (str)
        method  : - 0 : if you only want to like and comment without follow their account
                  - 1 : follow + like and comment
        """
        #to distinguish between just like-comment(0), with follow-comment-like(1)
        if method == 0:
            self.driver.get("https://www.instagram.com/"+account)
        
        #get the number of posts
        posts = self.driver.find_elements_by_class_name('-nal3 ')[0].text.split()[0]
        posts = int(Decimal(sub(r'[^\d.]', '', posts)))
        
        #click first pict
        pic = self.driver.find_element_by_class_name("_9AhH0")    
        pic.click()
        sleep(2)

        #click like button on the first pict
        like = self.driver.find_elements_by_class_name('wpO6b ')[2]
        like.click()

        #comment click (some comments have limited status)
        try:
            commentArea = self.driver.find_element_by_class_name('Ypffh')
            commentArea.click()
            sleep(2)
            commentArea = self.driver.find_element_by_class_name('Ypffh')
            commentArea.click()
            commentArea.send_keys(comments)
            commentArea.send_keys(Keys.ENTER)
            sleep(2)
        except:
            print("Limited Comment")

        #click next button        
        self.driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/div/a').click()
        print("Account: ", account, "like - comment pict-1 : done") 
        sleep(2)

        #like the first 3 posts
        if posts > 3:
            for i in range(2,4):
                #like photo
                try:
                    like = self.driver.find_element_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/section[1]/span[1]/button')
                    like.click()
                    sleep(2)
                except:
                    print("I can not fund this button")
                
                #next post handling
                if i != 3:
                    try:
                        nextPic = self.driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/div/a[2]')
                        nextPic.click()
                        print("Account: ", account, "like - comment pict-"+str(i)+"  : done") 
                        sleep(2)
                    except:
                        print("I the next button")
                        
        
    def start_follow(self, accounts, comments):
        """
        accounts : list of account we will follow (list of str)
        comments : list of comments (list of str)
        """
        driver = self.driver
        #looping itteration for each instagram account
        for idx, account in enumerate(accounts):
            #go to user account page
            driver.get("https://instagram.com/"+account+"/")
            sleep(random.randint(2,4))
            
            #get the follower number
            followers_element = self.driver.find_elements_by_class_name('-nal3 ')[1].text.split()[0]
            follower_number = int(Decimal(sub(r'[^\d.]', '', followers_element)))
            #if the number of followers like: 24K, 1.6M
            follower_character = ""
            try:
                follower_character = sub(r'[^\d.]', ' flaq', followers_element)
                follower_character = follower_character.split()[1]
            except:
                follower_character = ""
            
            #filter the account, we just follow an account which have more then 150 follower
            if (follower_number>150 or follower_character != ""):
                #There are 2 kinds of account : public (follow like comment) and private (just follow). They have different web structure
                #public account
                try:
                    #click follow button
                    driver.find_element_by_xpath("//button[@class='_5f5mN       jIbKX  _6VtSN     yZn4P   ']").click()
                    sleep(random.randint(2,3))

                    #call auto like-comment function
                    self.like_comment(account, comments[random.randint(0,3)])
                except:
                    #private account (just click follow button)
                    driver.find_element_by_xpath("//button[@class='sqdOP  L3NKy   y3zKF     ']").click()
                    sleep(random.randint(2,3))

                #randome activity to make the instagram doesn't suspect that we are bot (for every 5 posts)
                if idx % 5 == 0:
                    if random.randint(1,2) == 1:
                        driver.get("https://instagram.com/"+self.username+"/")
                    else:
                        driver.get("https://instagram.com/")
                    sleep(random.randint(5,10))
                  
        driver.get("https://instagram.com/"+self.username+"/")
        sleep(2)
        
    def start_unfollow(self, accounts):
        #accounts : list of account we want to unfollowe (list of str)
        driver = self.driver
        for account in accounts:
            #go to user account page
            driver.get("https://instagram.com/"+account+"/")
            #click icon friend
            driver.find_element_by_xpath("//button[@class='_5f5mN    -fzfL     _6VtSN     yZn4P   ']").click()
            sleep(2)
            
            #click unfollow button
            driver.find_element_by_xpath("//button[@class='aOOlW -Cab_   ']").click()
            sleep(random.randint(2,3))
        driver.get("https://instagram.com/"+self.username+"/")
        sleep(2)
        
    def start_dm(self, accounts, message):
        """
        accounts : list of account we will follow (list of str)
        message  : contents of the message that we will send (str)
        """
        for user_account in accounts:
            try:
                driver = self.driver
                #go to massage page 
                driver.get("https://www.instagram.com/direct/inbox/")
                #click send message
                driver.find_element_by_xpath("/html/body/div[1]/section/div/div[2]/div/div/div[2]/div/button").click()
                
                #fill the search box
                account_direct = driver.find_element_by_xpath("//input[@name='queryBox']")
                account_direct.clear()
                account_direct.send_keys(user_account) 
                sleep(2)

                #get the first account from the search list
                driver.find_element_by_xpath("/html/body/div[5]/div/div/div[2]/div[2]/div[1]").click()
                sleep(1)
                #klik next button
                driver.find_element_by_xpath("/html/body/div[5]/div/div/div[1]/div/div[2]").click()
                sleep(1)

                #send message
                try:
                    #click the textarea
                    commentArea = self.driver.find_element_by_xpath('/html/body/div[1]/section/div/div[2]/div/div/div[2]/div[2]/div/div[2]/div/div/div[2]/textarea')
                    commentArea.click()
                    sleep(2)
                    commentArea = self.driver.find_element_by_xpath('/html/body/div[1]/section/div/div[2]/div/div/div[2]/div[2]/div/div[2]/div/div/div[2]/textarea')
                    commentArea.click()
                    #fill the textarea
                    commentArea.send_keys(message)
                    #press enter
                    commentArea.send_keys(Keys.ENTER)
                    sleep(2)
                except:
                    print("Filed send")
            except:
                print("Filed DM to ",user_account)
            sleep(2)
        driver.get("https://instagram.com/"+self.username+"/")

## Scrape the data

### Login and get our account info

In [31]:
#xiena_ribuets
insta = login_instaloader('username','password')
followers, follows, not_follback, fans = get_followers_following(insta, 'irwan_afandi')

Account: irwan_afandi with 1163 followers, 1166 follow, 376 doesn't follback and 373 fans


### Who are they..? (followers, followed, not_follback, fans)

In [32]:
followers.head()

,account
0,cantika_furry
1,fiqriwicaksono
2,muhsatrio.go
3,naiilatulfadhiilah
4,dariswanj


In [33]:
follows.head()

,account
0,arvin_patria
1,fiqriwicaksono
2,avanputravan24
3,doktergigi_
4,lifeatshopee


In [38]:
pd.set_option('display.max_rows', not_follback.shape[0]+1)
n_follback = not_follback.reset_index()
n_follback.drop('index', axis=1, inplace =True)
print(n_follback)

                           account
0                     arvin_patria
1                   avanputravan24
2                      doktergigi_
3                     lifeatshopee
4                    qluesmartcity
5                  lifeattelkomsel
6                   bisnis.muda.id
7                   rakaminacademy
8                       nmonarizqa
9                          arsenal
10                    chris.korten
11                 kelasdata_iykra
12                     anakdata.id
13          transfermarkt_official
14                     halofina.id
15                   syeikh_muda22
16                        shvrarsy
17                   jagobahasacom
18                masak2dengannick
19                        kotakode
20    allah.is.the.most.merciful.1
21                         dtsense
22                     narasiodata
23                    codex.telkom
24                     fcbarcelona
25                     dioprayogii
26                       pacmannai
27                  

In [35]:
fans.head()

,account
0,cantika_furry
6,yt.mtfox
7,anggidwu1998
8,handsomefrog_bdg
18,equesttech1


In [47]:
#dataframe to list 
fans_array = fans.account.tolist()
fans_array

['cantika_furry',
 'yt.mtfox',
 'anggidwu1998',
 'handsomefrog_bdg',
 'equesttech1',
 'noonacraft.id',
 'binksstore53_testimoni',
 'techhub746',
 'klinik.essay',
 'dheanis.id',
 'studio.photo.qyta',
 'rhmnurs_',
 'dealova_hills',
 'jakartalabs',
 'ikhsan__nu',
 'nutribowl.id',
 'shindytri_',
 'adenascarf',
 '_banyupratama_',
 'auliavicky2905',
 'kumpultraderfx',
 'sjdhrc_',
 'hadinrp_propertybanyuwangi',
 'fadhli.rr',
 'thehubai',
 'makerdemy',
 'rustamapm',
 'smileyouss',
 'widhyanovhy',
 'wafazahida17',
 'wildan_dng',
 'niagaakademi',
 'dylan_tenor.781',
 'niagacorps',
 'gigastaufan',
 'bylenca',
 'donats_tjg',
 'pharis.prakoso',
 'delisamaudy',
 'jaket.keren_bandung',
 'juliberlalu',
 'pasundanight',
 'pakar_manageolshop',
 '0artdesign',
 'figuring.id',
 'yourtravel_mate',
 'cindylediasaputri',
 'celana_cargo_murah_kbm',
 'rajwaayunindya',
 'laper_baper30',
 'reparasielektroniksumedang',
 'annetees.id',
 'jualpelangsingcepat',
 'wahyu__z',
 'tokobukunadia',
 'dodik_haryadi08',
 'ani

## Start to run the Bot

### Login to our account

In [41]:
#our rundom comment
comments = ["Wow, I like your pict broh! Follback please :)", 
           "Keren pisan euy fotonya. Boleh follbacknya :D",
           "Cakep kali ini mah. Follback please :v",
           "Salam kenal kak, follbacknya boleh hehe"]

insta_bot = InstagramBot("username","password")
insta_bot.login()

### Follow account + like 3 post and leave a single comment to their post 

In [42]:
#follower 3.1m, private account, follower biasa, follower<150
insta_bot.start_follow(['tanboy_kun','__mllmllm18__', 'resepcemilan.unik', 'charvihijab'],comments)

Account:  tanboy_kun like - comment pict-1 : done
Account:  tanboy_kun like - comment pict-2  : done
Account:  resepcemilan.unik like - comment pict-1 : done
Account:  resepcemilan.unik like - comment pict-2  : done


### like 3 post and leave a single comment to their post (without follow)

In [122]:
comments = ["Wow, I like your pict broh! Follback please :)", 
           "Keren pisan euy fotonya. Boleh follbacknya :D",
           "Cakep kali ini mah. Follback please :v",
           "Salam kenal kak, follbacknya boleh hehe"]

insta_bot.like_comment('leo_edw',comments[random.randint(0,3)],0)

Account:  leo_edw like - comment pict-1 : done
Account:  leo_edw like - comment pict-2  : done


### Direct Message to some accounts

In [43]:
list_dm =['irwan_afandi', 'miralem_pjanic']
insta_bot.start_dm(list_dm, "Gan cakep kali sih awak nih :D")

### Unfollow some accounts

In [45]:
list_unfollow = ['sidomulyo_stable', 'pusatmikasurabaya']
insta_bot.start_unfollow(list_unfollow)